In [1]:
import numpy as np
import pandas as pd
import netCDF4
from netCDF4 import Dataset
from netCDF4 import MFDataset
import datetime
from pathlib import Path
import xarray as xr
from datetime import datetime

In [2]:
merra = xr.open_mfdataset('D:/Codes/IIA Project/Project Data/1.MERRA-2/*SUB.nc')

In [3]:
merra = merra.to_dataframe()

In [4]:
# Do merra.reset_index() to make lat, long and time as columns of the dataframe and change multiindex to single imndex

In [5]:
merra = merra.reset_index()

In [6]:
merra.head(5)

,lat,lon,time,PBLH
0,32.5,78.125,1980-01-01 00:30:00,632.503418
1,32.5,78.125,1980-01-01 01:30:00,1109.745239
2,32.5,78.125,1980-01-01 02:30:00,2237.738281
3,32.5,78.125,1980-01-01 03:30:00,2224.764893
4,32.5,78.125,1980-01-01 04:30:00,2078.818359


In [45]:
#merra['han'] = ((merra.PBLH[(merra['lat'] == 32.75) & (merra['lon'] == 78.75)])*0.0625) + ((merra.PBLH[(merra['lat'] == 32.75) & (merra['lon'] == 79.0625)]*0.25))/(0.0625+0.25)

In [7]:
han_1 = ((merra.PBLH[(merra['lat'] == 32.75) & (merra['lon'] == 78.75)])*0.0625).values
han_2 = ((merra.PBLH[(merra['lat'] == 32.75) & (merra['lon'] == 79.0625)]*0.25)).values
division_factor_for_interpolation = 0.0625 + 0.25

In [8]:
mer_1 = ((merra.PBLH[(merra['lat'] == 33.75) & (merra['lon'] == 78.4375)])*0.25).values
mer_2 = ((merra.PBLH[(merra['lat'] == 33.75) & (merra['lon'] == 78.75)]*0.0625)).values
division_factor_for_interpolation = 0.0625 + 0.25

In [9]:
interpolated_han_79_E = np.add(han_1, han_2)/division_factor_for_interpolation
interpolated_mer_78_dot_5_E = np.add(mer_1, mer_2)/division_factor_for_interpolation

In [10]:

date = pd.Series(merra['time'].unique()).apply(lambda x:x.date())
#unique attribute gives unique items from pd series in the same order


In [11]:
time = pd.Series(merra['time'].unique()).apply(lambda x:x.time())

In [15]:
final = pd.DataFrame(columns=['date','time','han_pblh','mer_pblh'],index = range(0,time.shape[0]))

In [16]:
final['time'] = time
final['date'] = date
final['han_pblh'] = interpolated_han_79_E/1000
final['mer_pblh'] = interpolated_mer_78_dot_5_E/1000

In [71]:
final.to_csv('D:/Codes/IIA Project/Output Data/merra_full.txt',index = False,sep = '\t',columns=['date','time','han_pblh','mer_pblh'])

In [19]:
final_nighttime = final[lambda df: (df.time >= datetime.strptime('18:00:00','%H:%M:%S').time()) | (df.time <= datetime.strptime('06:00:00','%H:%M:%S').time())]
final_daytime = final[lambda df: (df.time < datetime.strptime('18:00:00','%H:%M:%S').time()) & (df.time >= datetime.strptime('06:00:00','%H:%M:%S').time())]

In [21]:
avg_daytime = final_daytime[['date','han_pblh','mer_pblh']].groupby(['date']).mean()
avg_nighttime = final_nighttime[['date','han_pblh','mer_pblh']].groupby(['date']).mean()

In [20]:
avg_daily = final[['date','han_pblh','mer_pblh']].groupby(['date']).agg('mean')
median_daily = final.groupby(['date']).agg('median')
median_daytime = final_daytime.groupby(['date']).median()
median_nighttime = final_nighttime.groupby(['date']).median()

In [70]:
avg_daily.to_csv('D:/Codes/IIA Project/Output Data/merra_daily_avg.txt',sep='\t')
avg_daytime.to_csv('D:/Codes/IIA Project/Output Data/merra_daytime_avg.txt',sep='\t')
avg_nighttime.to_csv('D:/Codes/IIA Project/Output Data/merra_nighttime_avg.txt',sep='\t')

In [22]:
merra_daily = pd.merge(avg_daily,median_daily,how='inner',left_index=True,right_index=True,suffixes=('_avg','_median'))
merra_daytime = pd.merge(avg_daytime,median_daytime,how='inner',left_index=True,right_index=True,suffixes=('_avg','_median'))
merra_nighttime = pd.merge(avg_nighttime,median_nighttime,how='inner',left_index=True,right_index=True,suffixes=('_avg','_median'))

In [23]:
merra_daily.to_csv('D:/Codes/IIA Project/Output Data/merra_daily.txt',sep='\t')
merra_daytime.to_csv('D:/Codes/IIA Project/Output Data/merra_daytime.txt',sep='\t')
merra_nighttime.to_csv('D:/Codes/IIA Project/Output Data/merra_nighttime.txt',sep='\t')

In [24]:
final['year'] = final['date'].apply(lambda x: x.year)
final['month'] = final['date'].apply(lambda x: x.month)

In [25]:
avg_monthly = final.groupby(['year','month']).mean().reset_index([0,1])
median_monthly = final.group

In [68]:
avg_monthly.to_csv('D:/Codes/IIA Project/Output Data/merra_monthly_avg.txt',sep='\t',index=False)